In [3]:
search_for = 128
start_from = 0
threads = 10
thread = 2
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.015388965606689453
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 330
all cases: 866251507
type: [1, 1, 1, 1, 128] 128
cases of this type: 268435456
100000 44.60454356584033
200000 76.4520091986749
300000 72.30439806161641
400000 70.99299282050173
500000 71.31450059670499
600000 74.78634646776044
700000 70.96846626800131
800000 70.74534187832415
900000 70.56164598007318
1000000 69.55081517278244
1100000 20.5190556291497
1200000 70.65024052496263
1300000 68.85668320564481
1400000 68.492479903445
1500000 68.77777016743413
1600000 72.68324650032373
1700000 70.26796581315851
1800000 67.81827683193907
1900000 70.06142597325939
2000000 69.87205952232524
2100000 54.93069069020222
2200000 36.89462906010307
2300000 66.64994035858363
2400000 66.00852717888662
2500000 63.992305895235035
2600000 67.32436165665672
2700000 66.10825490790994
2800000 65.51868176219928
2900000 64.87735023981588
3000000 65.51187272600066
310

30100000 75.19719927251819
30200000 73.77541825920463
30300000 70.43991876738423
30400000 58.50379872925302
30500000 41.77763455551504
30600000 66.20819343052277
30700000 75.29111575740865
30800000 69.91524411490855
30900000 74.65563224733648
31000000 76.49789911699733
31100000 78.58299888306423
31200000 73.71180434893539
31300000 36.213013500303575
31400000 74.95579266626443
31500000 75.47372325219136
31600000 74.05542503077628
31700000 59.73842336283181
31800000 43.82036052433059
31900000 72.56825138199109
32000000 71.46056303525857
32100000 75.88744805240172
32200000 66.06209944993331
32300000 65.51801289524347
32400000 62.68610459472891
32500000 51.99426074917671
32600000 42.40457651163396
32700000 66.50960424995837
32800000 70.20461856979716
32900000 68.06948104432261
33000000 72.97147139554191
33100000 73.7296357372287
33200000 72.22303582757891
33300000 61.187874718371546
33400000 44.672379885871074
33500000 74.37097307325197
33600000 73.58367237830181
33700000 75.54214961663835

60600000 75.40519939034466
60700000 82.08172405460927
60800000 75.33001255387151
60900000 73.78260529447854
61000000 73.74631120239184
61100000 71.54790263998962
61200000 80.1619926693623
61300000 51.996495066064504
61400000 48.606946367669906
61500000 64.64897401727393
61600000 64.79670450812968
61700000 69.35218925647604
61800000 64.32288537114495
61900000 32.44724392515628
62000000 63.18274886884077
62100000 62.788243784621486
62200000 68.87428314015155
62300000 64.7387703152609
62400000 70.35203672717257
62500000 33.504104022796646
62600000 71.23780578879484
62700000 74.75403407309578
62800000 73.88164205942928
62900000 78.29536175213042
63000000 75.87335252916732
63100000 75.68902449499322
63200000 74.17727960738637
63300000 78.69977036155815
63400000 61.03955273214894
63500000 45.07545989070091
63600000 69.42839556478344
63700000 68.08598585274588
63800000 69.3363675294571
63900000 72.99494311672244
64000000 28.420354218320412
64100000 61.95268469217488
64200000 67.15687295669845

91000000 46.12608473422205
91100000 63.631805509372576
91200000 58.37040436601171
91300000 23.78039468552426
91400000 62.38255206778651
91500000 62.45208665907286
91600000 39.30768203396452
91700000 60.432752127493686
91800000 61.87704990117354
91900000 65.49768867028696
92000000 63.57058977262009
92100000 68.71127894444591
92200000 68.54850282223748
92300000 70.43291214400675
92400000 72.86868830688763
92500000 72.40978566520342
92600000 71.05833584479639
92700000 69.94764802543986
92800000 65.59577637492242
92900000 64.9552089857995
93000000 45.669098985168425
93100000 47.10232332095389
93200000 66.0107127063198
93300000 57.90217950106722
93400000 26.976828583562774
93500000 64.28243382502964
93600000 63.484547895046745
93700000 28.808136349090514
93800000 66.81374978669925
93900000 64.92230557005078
94000000 64.32037751048914
94100000 70.01114705716512
94200000 73.00083974558193
94300000 71.77161509807769
94400000 73.6998893137699
94500000 70.6090397277371
94600000 67.51429471862855

120500000 33.77787222108362
120600000 31.71548861572682
120700000 30.98430951956047
120800000 43.240558307325145
120900000 60.683429396365256
121000000 60.60600128453116
121100000 58.379779218576324
121200000 61.73541412719733
121300000 61.04301471191595
121400000 65.55657964406825
121500000 62.87085994047685
121600000 67.27180066599456
121700000 68.3418600966062
121800000 67.46701284317389
121900000 63.88881999892545
122000000 64.05237763401269
122100000 64.95216484401952
122200000 62.72846043846311
122300000 59.84902872238355
122400000 57.87380722918487
122500000 59.29515587781462
122600000 42.0896960546262
122700000 21.511397990102108
122800000 24.647418474054476
122900000 52.407270515025736
123000000 63.16687675839318
123100000 60.32809410687482
123200000 61.75371997958251
123300000 64.24391720083496
123400000 64.99570441776542
123500000 64.05126673710726
123600000 62.04864470703636
123700000 67.5143546794276
123800000 66.22667685932913
123900000 58.871828673607894
124000000 62.200

...

best so far: 0
type: [1, 2, 16, 1, 4] 128
cases of this type: 131072
799800000 0.24410546232225827
799900000 0.2928136439499299
best so far: 0
type: [1, 2, 16, 2, 2] 128
cases of this type: 65536
best so far: 0
type: [1, 2, 16, 4, 1] 128
cases of this type: 32768
800000000 0.24991734490451745
best so far: 0
type: [1, 2, 32, 1, 2] 128
cases of this type: 32768
best so far: 0
type: [1, 2, 32, 2, 1] 128
cases of this type: 16384
best so far: 0
type: [1, 2, 64, 1, 1] 128
cases of this type: 8192
best so far: 0
type: [1, 4, 1, 1, 32] 128
cases of this type: 4194304
800100000 0.5801478581966022
800200000 1.6187525440258044
800300000 1.6640498888692152
800400000 1.8103078582552306
800500000 1.0210931302492081
800600000 0.9563716121006264
800700000 0.9685813865594413
800800000 0.8170985289290688
800900000 0.8960361237338371
801000000 0.8731473354648716
801100000 0.78963962121285
801200000 0.9672351230628033
801300000 1.130986456492508
801400000 1.0894408250405014
801500000 1.4280875441245973
8

813400000 0.9911949854613721
813500000 1.1612248269715588
813600000 0.8589678726523796
813700000 0.9054665427968384
813800000 1.1560220902745333
813900000 1.576377598923397
814000000 1.6140456875875175
814100000 1.0003864349459541
814200000 0.9720696566742109
814300000 0.8828036722893914
814400000 1.3110276290147245
814500000 1.3655826826187973
814600000 1.5010163374988454
814700000 1.3263321193817055
814800000 0.8720845406596528
814900000 0.9574945112767815
815000000 0.9858146765268379
815100000 0.8955889459498881
815200000 0.9198988641310513
815300000 0.7991925426199469
815400000 0.9209334088083193
815500000 1.390962944556411
815600000 1.1240985655585924
815700000 0.8945911292324973
815800000 1.0900696076394976
815900000 1.2166645946845704
816000000 0.9223416181512271
816100000 0.9711108653229454
816200000 1.4631555390708506
816300000 1.2607045187590744
816400000 1.3425125259959003
816500000 1.4708953902869788
816600000 1.3981638718786704
816700000 1.6515084485352516
816800000 1.2575

841100000 0.26577349700458114
841200000 0.22460396956413187
841300000 0.25403084249057767
841400000 0.32622785061673654
841500000 0.3094490958602382
841600000 0.2375896620188018
841700000 0.25053980333451037
841800000 0.16018464358149037
best so far: 0
type: [2, 1, 1, 8, 8] 128
cases of this type: 2097152
841900000 0.4228983828229129
842000000 0.2827252961343984
842100000 0.18120473192850284
842200000 0.3296637647182663
842300000 0.18214252617976795
842400000 0.20836677608551776
842500000 0.3138326896465659
842600000 0.23521703809856176
842700000 0.3657740761645688
842800000 0.21310453430095316
842900000 0.3207402859279911
843000000 0.4558447357366019
843100000 0.31141021662154994
843200000 0.3678800671371957
843300000 0.29283604483080306
843400000 0.16489374799318116
843500000 0.23673726671060452
843600000 0.17078106998659637
843700000 0.18961717967623473
843800000 0.22734088155182663
843900000 0.27676869092364376
best so far: 0
type: [2, 1, 1, 16, 4] 128
cases of this type: 1048576
8

861300000 0.04034003873609437
best so far: 0
type: [2, 2, 2, 2, 8] 128
cases of this type: 262144
861400000 0.049039437791615716
861500000 0.035575208031761646
861600000 0.023171213693805533
best so far: 0
type: [2, 2, 2, 4, 4] 128
cases of this type: 131072
861700000 0.03475950364838839
best so far: 0
type: [2, 2, 2, 8, 2] 128
cases of this type: 65536
861800000 0.028841179944355623
best so far: 0
type: [2, 2, 2, 16, 1] 128
cases of this type: 32768
best so far: 0
type: [2, 2, 4, 1, 8] 128
cases of this type: 131072
861900000 0.02922054677115811
best so far: 0
type: [2, 2, 4, 2, 4] 128
cases of this type: 65536
862000000 0.021486672996407746
best so far: 0
type: [2, 2, 4, 4, 2] 128
cases of this type: 32768
best so far: 0
type: [2, 2, 4, 8, 1] 128
cases of this type: 16384
862100000 0.017632958921543755
best so far: 0
type: [2, 2, 8, 1, 4] 128
cases of this type: 32768
best so far: 0
type: [2, 2, 8, 2, 2] 128
cases of this type: 16384
best so far: 0
type: [2, 2, 8, 4, 1] 128
cases of 

866100000 0.0009239688851614792
best so far: 0
type: [8, 1, 1, 2, 8] 128
cases of this type: 32768
best so far: 0
type: [8, 1, 1, 4, 4] 128
cases of this type: 16384
best so far: 0
type: [8, 1, 1, 8, 2] 128
cases of this type: 8192
best so far: 0
type: [8, 1, 1, 16, 1] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 2, 1, 8] 128
cases of this type: 16384
best so far: 0
type: [8, 1, 2, 2, 4] 128
cases of this type: 8192
866200000 0.0003231077828586101
best so far: 0
type: [8, 1, 2, 4, 2] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 2, 8, 1] 128
cases of this type: 2048
best so far: 0
type: [8, 1, 4, 1, 4] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 4, 2, 2] 128
cases of this type: 2048
best so far: 0
type: [8, 1, 4, 4, 1] 128
cases of this type: 1024
best so far: 0
type: [8, 1, 8, 1, 2] 128
cases of this type: 1024
best so far: 0
type: [8, 1, 8, 2, 1] 128
cases of this type: 512
best so far: 0
type: [8, 1, 16, 1, 1] 128
cases of this type: 256
best so f